---



# Unit 2 - Part 2a: The Anatomy of a Prompt

## 1. Introduction: Stochasticity (Randomness)

Why does the AI give different answers? Because it is **Stochastic** (Random).

It predicts the NEXT TOKEN based on probability.

### Visualizing the Prediction
Input: `"The sky is..."`

| Word | Probability | Selected? (Temp=0) | Selected? (Temp=1) |
|------|-------------|--------------------|--------------------|
| Blue | 80% | ✅ | ❌ |
| Gray | 15% | ❌ | ✅ |
| Green| 1% | ❌ | ❌ |

Prompt Engineering is the art of **manipulating these probabilities**.

## Setting Up the Environment

First, let's install the required packages.

In [1]:
%pip install --upgrade --quiet python-dotenv langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.9 MB/s eta 0:00:00


In [2]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


## 2. The CO-STAR Framework (simplified)

A good prompt usually has:
1.  **C**ontext (Who are you? Who acts?)
2.  **O**bjective (What is the task?)
3.  **S**tyle (Formal? Funny?)
4.  **T**one (Empathetic? Direct?)
5.  **A**udience (Who is reading this?)
6.  **R**esponse Format (JSON? List?)

Let's compare a **Lazy Prompt** vs a **CO-STAR Prompt**.

In [3]:
# The Task: Reject a candidate for a job.
task = "Write a rejection email to a candidate named Sarah who applied for a Senior Machine Learning Engineer position."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Subject: Update on Your Senior Machine Learning Engineer Application at [Company Name]

Dear Sarah,

Thank you for your interest in the Senior Machine Learning Engineer position at [Company Name] and for taking the time to interview with our team. We truly appreciate you sharing your experience and qualifications with us.

We received a large number of highly qualified applicants for this role, and after careful consideration, we have decided to move forward with other candidates whose experience and skills were a closer match for the specific requirements of this position at this time.

This was a very competitive search, and we truly appreciate your interest in joining our team. We wish you the very best in your job search and future endeavors.

Sincerely,

[Your Name/Hiring Manager Name]
[Your Title/HR Department]
[Company Name]


## 3. Hallucination vs. Creativity

Did the model make up a reason?
Since we didn't give it facts, it **Predicted the most likely reason** (Usually "Experience" or "Volume of applications").

**This is NOT a bug.** It is a feature. The model is *completing the pattern* of a rejection email.

In [4]:
structured_prompt = """
# Context
You are an HR Manager at a quirky AI startup called 'NeuralNinja' that builds AI tools for developers.

# Objective
Write a rejection email to a candidate named Alex Chen who applied for a Senior ML Engineer role.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role requirements evolved'.
3. Sign off with 'Keep innovating'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Dear Alex Chen,

Thank you for your interest in the Senior ML Engineer role at NeuralNinja. While your qualifications are impressive, the role requirements evolved during our process. We wish you the best in your job search.

Keep innovating,
NeuralNinja HR


## 4. Key Takeaway: Ambiguity is the Enemy

Every piece of information you leave out is a gap the model MUST fill with probability.
- If you don't say "Be brief", it picks the most probable length (Avg email length).
- If you don't say "Be rude", it picks the most probable tone (Polite/Neutral).

## Assignment

Write a structured prompt to generate a **Python Function**.
- **Context:** You are a Senior Python Dev.
- **Objective:** Write a function to reverse a string.
- **Constraint:** It must use recursion (no slicing `[::-1]`).
- **Style:** Include detailed docstrings.

In [5]:
# Assignment Solution

assignment_prompt = """
# Context
You are a Senior Python Developer at a tech company, known for writing clean, well-documented code.

# Objective
Write a Python function to reverse a string.

# Constraints
1. MUST use recursion (no slicing like [::-1]).
2. Include comprehensive docstrings with:
   - Function description
   - Args section
   - Returns section
   - Example usage
3. Add type hints.
4. Handle edge cases (empty string).

# Style
Professional, production-ready code with detailed documentation.

# Output Format
Only the Python code, no explanations outside the docstring.
"""

print("--- SOLUTION ---")
print(llm.invoke(assignment_prompt).content)

--- SOLUTION ---
```python
def reverse_string_recursive(s: str) -> str:
    """
    Reverses a given string using a recursive approach.

    This function takes an input string and returns a new string with its
    characters in reverse order. It operates by recursively processing the
    substring excluding the first character, and then appending the first
    character to the result of that recursive call.

    Args:
        s (str): The input string to be reversed.

    Returns:
        str: The reversed string.

    Examples:
        >>> reverse_string_recursive("hello")
        'olleh'
        >>> reverse_string_recursive("Python")
        'nohtyP'
        >>> reverse_string_recursive("")
        ''
        >>> reverse_string_recursive("a")
        'a'
        >>> reverse_string_recursive("racecar")
        'racecar'
    """
    # Base case: If the string is empty or contains a single character,
    # it is already reversed. This handles the empty string edge case
    # and the te

## Solution Explanation

**Key CO-STAR Elements Used:**
1. **Context:** Established expertise level (Senior Dev)
2. **Objective:** Clear task (reverse string)
3. **Style:** Professional, production-ready
4. **Constraints:** Specific technical requirements (recursion, type hints, edge cases)
5. **Response Format:** Code only, no extra text

This structured approach ensures the LLM generates exactly what we need!

---



# Unit 2 - Part 2b: Zero-Shot to Few-Shot

## 1. Introduction: In-Context Learning

How does the model learn without training?
This is called **In-Context Learning**.

### The Attention Mechanism (Flowchart)
When you ask a question, the model "looks back" at the previous text to find patterns.

```mermaid
graph TD
    Input[Current Input: 'Angry + Hungry'] -->|Attention Query| History
    subgraph History [The Prompt Examples]
        Ex1[Ex1: Breakfast + Lunch = Brunch]
        Ex2[Ex2: Chill + Relax = Chillax]
    end
    History -->|Pattern Found: Mix words & define| Prediction[Output: Hangry]
```

In [6]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

## 2. Zero-Shot (No Context)

The model relies purely on its training data.

In [7]:
prompt_zero = "Combine 'Work' and 'Vacation' into a creative portmanteau word and define it."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: Here are a few creative portmanteaus combining 'Work' and 'Vacation', each with a definition:

1.  **Workcation**
    *   **Definition:** A trip where an individual travels to a new location, often a leisure destination, and combines their professional work responsibilities with elements of a vacation, such as exploring the area, relaxing, or enjoying local amenities, typically while working remotely.
    *   *Example:* "After a stressful quarter, Sarah booked a two-week **workcation** in Bali, spending mornings on her laptop and afternoons surfing."

2.  **Bliss-ness**
    *   **Definition:** The harmonious integration of professional duties with personal enjoyment and relaxation, typically experienced when working remotely from a desirable vacation location, aiming for a state of productive contentment.
    *   *Example:* "Our team's new remote policy encourages **bliss-ness**, letting us find our perfect balance between deadlines and downtime."

3.  **Taskation**
    *   

## 3. Few-Shot (Pattern Matching)

We provide examples. The Attention Mechanism attends to the **Structure** (`Input -> Output`) and the **Tone** (Sarcasm).

In [9]:
prompt_few = """
Combine words into a creative portmanteau. Give a witty, sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink mimosas at 11 AM and call it 'cultured')

Input: Smoke + Fog
Output: Smog (Nature's way of punishing cities for having too many cars)

Input: Glamour + Camping
Output: Glamping (Pretending you love nature while sleeping in a $500/night tent with WiFi)

Input: Work + Vacation
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Workation (The modern delusion that you can escape the office by just moving your laptop to a beach.)


## 4. Critical Analysis

If you provide **bad examples**, the model will learn the **bad pattern**.
This is why Data Quality in your prompt is just as important as code quality.

---



# Unit 2 - Part 2c: Advanced Templates & Theory

## 1. Theory: Engineering vs. Training

### Hard Prompts (Prompt Engineering)
- **What:** You change the text input.
- **Cost:** Cheap, fast, easy to iterate.
- **Use Case:** Prototyping, General tasks.

### Soft Prompts (Fine Tuning)
- **What:** You change the model's internal weights (mathematically).
- **Cost:** Expensive, slow, needs data.
- **Use Case:** Domain specificity (Medical, Legal), Behavioral change.

In [10]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

## 2. Dynamic Few-Shotting

If you have 1000 examples, you can't fit them all in the context window.
We use a **Selector** to pick the best ones.

### The Selector Flow (Flowchart)
```mermaid
graph LR
    Input[User Input] -->|Semantic Search| Database[Example Database]
    Database -->|Top 3 Matches| Selector
    Selector -->|Inject| Prompt
```

In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
import time

# 1. Our Database of Examples
examples = [
    {"input": "The server crashed again.", "output": "We are experiencing an unplanned service interruption."},
    {"input": "This code is garbage.", "output": "The current implementation has room for optimization."},
    {"input": "The deadline is impossible.", "output": "The timeline presents ambitious delivery expectations."},
    {"input": "Nobody uses this feature.", "output": "This feature shows minimal user engagement metrics."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corporate-Speak Translator for tech companies. Transform blunt statements into diplomatic, professional language."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm | StrOutputParser()

# Test with multiple examples (with rate limiting and error handling)
test_inputs = [
    "This meeting was a waste of time.",
    "The design looks terrible.",
    "We're way over budget."
]

for i, test_input in enumerate(test_inputs):
    try:
        print(f"Input: {test_input}")
        print(f"Output: {chain.invoke({'text': test_input})}")
        print()

        # Add delay between requests to avoid rate limits (except for last one)
        if i < len(test_inputs) - 1:
            time.sleep(2)  # Wait 2 seconds between requests

    except Exception as e:
        if "RESOURCE_EXHAUSTED" in str(e) or "429" in str(e):
            print(f"⚠️ Quota limit reached! You've hit your daily API limit.")
            print(f"💡 Wait for your quota to reset or upgrade your plan.")
            print(f"🔗 Check usage: https://ai.dev/rate-limit")
            break
        else:
            print(f"❌ Error: {e}")
            break

Input: This meeting was a waste of time.
Output: I'm keen to ensure our future collaborative sessions are structured for optimal efficiency and actionable outcomes.

Input: The design looks terrible.
Output: The current visual aesthetic presents opportunities for refinement.

Input: We're way over budget.
Output: Our financial resource utilization requires optimization.



## 3. Analysis

Using `FewShotChatMessagePromptTemplate` creates a clean separation between instructions and data. This helps the Attention Mechanism focus on the right things.